In [33]:
# Cell 1: Imports and Environment Setup
import os
import requests
from dotenv import load_dotenv

# Import ADK Components
from google.adk.agents import Agent
from google.adk.models.google_llm import Gemini
from google.adk.tools import google_search, FunctionTool
from google.genai import types
from google.adk.tools import google_search


print("✅ Libraries and Components Imported.")

✅ Libraries and Components Imported.


In [34]:
import os
from dotenv import load_dotenv

# Load variables from .env file
load_dotenv()

try:
# Set the key for the Google Generative AI SDK
    os.environ['GEMINI_API_KEY'] = os.getenv('GEMINI_API_KEY')
    EDAMAM_URL = "https://api.edamam.com/api/nutrition-data"
    APP_ID = os.getenv('EDAMAM_APP_ID')
    APP_KEY = os.getenv('EDAMAM_APP_KEY')
    print("Environment variables loaded. You can now use the Google GenAI SDK.")
except Exception as e:
    print("🔑 Authentication Error: Please make sure you have added 'GEMINI_API_KEY' to your Kaggle secrets. Details: {e}")


Environment variables loaded. You can now use the Google GenAI SDK.


In [35]:
retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1, # Initial delay before first retry (in seconds)
    http_status_codes=[429, 500, 503, 504] # Retry on these HTTP errors
)
print(" Retry Config Loaded.")

 Retry Config Loaded.


In [36]:
# --- TOOL 1: Macro Lookup (with API Retry and Search Fallback) ---
def macro_lookup(food_item_and_weight: str) -> dict:
    """
    Looks up estimated Calories (kcal) and Protein (grams) using the Edamam API.
    Includes built-in query cleaning and a direct instruction to the LLM to 
    initiate a Google Search if API data is not found.
    """
    if not (APP_ID and APP_KEY):
        return {"status": "error", "message": "API keys are missing. Cannot perform lookup."}

    # Extract weight and item name using regex (for reliable API parsing)
    match = re.search(r'(\d+)\s*(g|oz|gram|ounce)s?\s*of\s*(.+)', food_item_and_weight, re.IGNORECASE)
    
    if match:
        weight = match.group(1)
        unit = match.group(2)
        food_item = match.group(3).strip()
        api_base_query = f"{weight} {unit}s {food_item}"
    else:
        api_base_query = food_item_and_weight # Use as-is if parsing fails

    # 1. Define Search/Retry Queries
    queries_to_try = [
        api_base_query, 
        api_base_query.replace("grilled", "").replace("cooked", "").replace("roasted", "").strip() 
    ]
    
    # 2. Execute API Calls with Retry
    for api_query in queries_to_try:
        params = {"app_id": APP_ID, "app_key": APP_KEY, "ingr": api_query}
        try:
            response = requests.get(EDAMAM_URL, params=params, timeout=10)
            response.raise_for_status()
            data = response.json()
            
            if data.get('totalNutrients') and data.get('calories') > 0:
                # SUCCESS: API data found
                calories = int(data.get('calories', 0))
                protein_g = data['totalNutrients'].get('PROCNT', {}).get('quantity', 0)
                
                return {
                    "status": "success",
                    "message": "Based on a precise API lookup.",
                    "calories": calories,
                    "protein_g": round(protein_g, 1),
                    "query_used": api_query
                }
        except requests.exceptions.RequestException:
            continue
    
    # 3. FALLBACK TRIGGER (If all API attempts fail)
    search_query = f"macro estimate for {food_item_and_weight} in grams"
    
    return {
        "status": "fallback_needed",
        "search_query": search_query,
        "message": "API data unavailable. The model must call Google Search with the provided search_query to find a text estimate."
    }

print("Custom tool : 1 defined")

Custom tool : 1 defined


In [37]:
def recipe_finder(ingredients: list[str]) -> dict:
    """
    Searches the Edamam Recipe API for a high-protein meal using the provided 
    ingredients. Use this when the user lists ingredients they have on hand.
    
    Args:
        ingredients: A list of ingredients the user has available.
        
    Returns:
        A dictionary containing the 'meal_name', 'source', 'url', and a summary of macros.
    """
    
    if not (APP_ID and APP_KEY):
        return {"status": "error", "message": "Edamam API keys are not loaded for recipe search."}
        
    # Join ingredients into a single search query string
    query_string = ", ".join(ingredients)
    
    params = {
        "app_id": APP_ID,
        "app_key": APP_KEY,
        "type": "public",  # Required for V2 API
        "q": query_string, # The ingredient list
        "health": "high-protein", # Crucial filter for your diet coach goal
        "limit": 1
    }
    
    try:
        response = requests.get(EDAMAM_RECIPE_URL, params=params, timeout=10)
        response.raise_for_status() 
        data = response.json()
        
        # Check for search results
        if data.get('hits') and len(data['hits']) > 0:
            recipe = data['hits'][0]['recipe']
            
            # Extract main nutritional info for the agent to synthesize
            calories = round(recipe['calories'], 0)
            protein = round(recipe['totalNutrients']['PROCNT']['quantity'], 1)
            
            return {
                "status": "success",
                "meal_name": recipe['label'],
                "source": recipe['source'],
                "url": recipe['url'],
                "estimated_macros": f"{calories} kcal, {protein}g Protein"
            }
        else:
            return {"status": "info", "message": f"Could not find a high-protein recipe matching ingredients: {query_string}. Advise the user to simplify the list or add a core protein source."}
            
    except requests.exceptions.RequestException as e:
        return {"status": "api_error", "message": f"Network or API call error during recipe search: {e}"}

print(" Custom Tool :2  Defined.")

 Custom Tool :2  Defined.


In [38]:
def protein_sweet_lookup(excluded_ingredients: list[str] = None) -> dict:
    """
    Searches the Edamam Recipe API for a dynamic, high-protein, low-sugar sweet treat
    recipe, avoiding any ingredients listed in the 'excluded_ingredients'.
    
    Args:
        excluded_ingredients: A list of ingredients to explicitly avoid.
        
    Returns:
        A dictionary containing the 'sweet_name', 'source', 'url', and a summary of macros.
    """
    
    if not (APP_ID and APP_KEY):
        return {"status": "error", "message": "Edamam API keys are not loaded for recipe search."}
        
    query_string = "protein low sugar dessert OR protein baked snack"
    
    params = {
        "app_id": APP_ID,
        "app_key": APP_KEY,
        "type": "public",
        "q": query_string,
        "health": ["sugar-conscious", "low-sugar", "high-protein"], 
        "limit": 1
    }
    
    # CRUCIAL FIX: Add the exclusion list to the parameters
    if excluded_ingredients:
        # Edamam uses 'excluded' parameter for ingredients to avoid
        params['excluded'] = excluded_ingredients
        
    # --- (The rest of the API call and result extraction logic remains the same) ---

    try:
        response = requests.get(EDAMAM_RECIPE_URL, params=params, timeout=10)
        response.raise_for_status() 
        data = response.json()
        
        # Check for search results and extract data...
        if data.get('hits') and len(data['hits']) > 0:
            # ... (Extraction logic remains the same) ...
            recipe = data['hits'][0]['recipe']
            calories = round(recipe['calories'], 0)
            protein = round(recipe['totalNutrients']['PROCNT']['quantity'], 1)
            
            return {
                "status": "success",
                "sweet_name": f"High-Protein Substitute: {recipe['label']}",
                "suggestion_message": "Stay consistent! Here is a great, low-sugar alternative:",
                "source": recipe['source'],
                "url": recipe['url'],
                "estimated_macros": f"{calories} kcal, {protein}g Protein"
            }
        else:
            return {"status": "info", "message": "Could not find a high-protein sweet recipe matching your criteria. Try different ingredients or search terms."}
            
    except requests.exceptions.RequestException as e:
        return {"status": "api_error", "message": f"Network or API call error: {e}"}
print(" Custom Tool:3 Defined.")

 Custom Tool:3 Defined.


In [39]:
# Assuming google_search is available in TOOL_LIST

def search_estimator(food_item_and_weight: str) -> dict:
    """
    MANDATORY STEP: Searches Google for the macro estimate of a food item 
    before the macro_lookup tool is called.
    """
    search_query = f"calories and protein in {food_item_and_weight} raw text"
    
    # 1. Instruct the LLM to run the search tool
    # The LLM will execute this and return the search results next turn.
    return {
        "status": "search_requested",
        "search_query": search_query,
        "message": "Use the Google Search tool with the search_query to find macro estimate."
    }

print("ok")

ok


In [40]:
# Cell 4: Agent Definition

# 🎯 Wrap the functions as tools for the ADK
macro_lookup_tool = FunctionTool(macro_lookup)
recipe_finder_tool = FunctionTool(recipe_finder)
protein_sweet_lookup_tool = FunctionTool(protein_sweet_lookup)

root_agent = Agent(
    name="My_AI_Diet_Coach",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    description="An expert AI Diet Coach for personalized macro and calorie goal guidance.",
    instruction="""You are a supportive, expert, and highly precise Diet Coach AI.

**MANDATORY SEQUENCE (NO EXTERNAL API):** For ANY calorie or macro query, you MUST follow this sequence:
1. **Tool Call (Search Formatter):** Call the **macro_lookup** tool immediately with the precise or estimated food item and weight (e.g., '40g cooked chicken breast').
2. **Execute Tool:** Execute the macro_lookup tool.
3. **Synthesis & Fallback:** Read the macro_lookup tool's output. 
   - If 'status' is 'success', synthesize the final answer.
   - If 'status' is 'fallback_needed', you MUST execute the **google_search** tool using the provided 'search_query'.
4. **Final Answer:** Synthesize the final, estimated answer based ONLY on the data received.

**Estimation Protocol (Gram Metrics):** If the quantity is vague (e.g., 'medium piece'), assume a medium piece is 170g and call the tool with that estimate (e.g., '170g chicken breast').

**Tool List Definition:**
""",
    tools=[ 
        macro_lookup_tool,
        recipe_finder_tool,
        protein_sweet_lookup_tool,
        google_search # The built-in Google Search Tool
    ] 
)

print("✅ Root Agent defined with all tools.")

✅ Root Agent defined with all tools.


In [41]:
from google import genai 
from google.genai import types

# Assuming client, ESTIMATES, SYSTEM_INSTRUCTION, and TOOL_LIST are defined globally

def execute_agent_logic(prompt_text: str):
    """
    Runs the prompt through Gemini, augmenting the prompt, and handling 
    multi-step tool calls (API -> Search Fallback) in a persistent loop.
    """
    
    print(f"\n--- Running Prompt: {prompt_text} ---")
    
    # --- 1. PROMPT AUGMENTATION (Estimate Hint) ---
    augmented_prompt = prompt_text
    vague_term_detected = False
    
    # Check for vague terms and augment the prompt
    for term, estimate in ESTIMATES.items():
        if term in prompt_text.lower():
            # Create a strong hint based on the first match
            augmented_prompt = f"NOTE TO SELF: The vague term '{term}' MUST be estimated as {estimate}. "
            augmented_prompt += "Your instruction is to use this estimate for the tool call. "
            augmented_prompt += prompt_text
            vague_term_detected = True
            break
            
    if vague_term_detected:
        print(f"-> Sending Augmented Prompt (Estimate Hint Included)")
    
    # --- 2. INITIAL CALL SETUP ---
    
    # Start the conversation history with the (potentially augmented) user prompt
    current_contents = [types.Content(role="user", parts=[types.Part(text=augmented_prompt)])]

    max_turns = 5 
    turn = 0
    response = None 

    # --- 3. TOOL CALLING LOOP (The Core Logic) ---
    
    while turn < max_turns:
        turn += 1
        
        # Call the model with the current conversation history
        response = client.models.generate_content(
            model='gemini-2.5-flash',
            contents=current_contents,
            config=types.GenerateContentConfig(
                system_instruction=SYSTEM_INSTRUCTION,
                tools=TOOL_LIST
            )
        )

        # EXIT CONDITION 1: Model returned a final text response (SUCCESS)
        if response.text:
            break

        # Tool Calling Logic
        if response.function_calls:
            call = response.function_calls[0]
            tool_name = call.name
            tool_func = globals().get(tool_name)
            
            print(f"-> Turn {turn}: Model requested tool call: {tool_name}")

            # 1. Add Model's request to history (REQUIRED before executing)
            current_contents.append(response.candidates[0].content)

            if tool_func:
                args = dict(call.args)
                tool_output = tool_func(**args)
                
                # --- CHECK FOR FALLBACK TRIGGER (If API Fails) ---
                if tool_name == 'macro_lookup' and tool_output.get('status') == 'fallback_needed':
                    print("-> API failed! Synthesizing Google Search request...")
                    
                    # Synthesize the search request using the query from the failed tool's output
                    search_query = tool_output['search_query']
                    
                    # Tool output part instructing the LLM to search next
                    tool_output_part = types.Part.from_function_response(
                        name="google_search", 
                        response={"query": search_query} # LLM will read this and execute search next
                    )
                else:
                    # Success or non-macro tool output
                    tool_output_part = types.Part.from_function_response(name=tool_name, response=tool_output)

                # 2. Add Tool's output (either API data or Search Instruction) to history
                current_contents.append(types.Content(role="tool", parts=[tool_output_part]))
            
            else:
                print(f"-> Error: Unknown tool {tool_name}. Breaking loop.")
                break 

        else:
            # EXIT CONDITION 2: Model returned neither text nor function call (FAILURE)
            print("-> Model loop terminated without result or function call. Breaking.")
            break 
            
    # --- 4. FINAL RESPONSE ---
    if response and response.text:
        # The Final Agent Response is synthesized after the search or API call is successful
        print(f"\n✅ Final Agent Response:\n{response.text}")
    else:
        print("\n❌ Execution Failed: Model did not produce a final text response after loop.")
print("agent logic execution works")

agent logic execution works


In [42]:

# --- Execute Tests ---

# Test 1: Live Macro Lookup (Triggers the macro_lookup function)
execute_agent_logic(
    prompt_text="What are the calories and protein in 40g of chicken breast"
)



--- Running Prompt: What are the calories and protein in 40g of chicken breast ---


NameError: name 'ESTIMATES' is not defined

In [12]:
# Test 2: Cravings Handler (Triggers the protein_sweet_lookup function)
execute_agent_logic(
    prompt_text="I have rice, 2 eggs, celery, zuchini and tomatoes, and soy sauce and mayonaise. what ccan i make."
)


--- Running Prompt: I have rice, 2 eggs, celery, zuchini and tomatoes, and soy sauce and mayonaise. what ccan i make. ---


NameError: name 'ESTIMATES' is not defined

In [2]:

# Test 3: Cravings Handler (Triggers the protein_sweet_lookup function)
execute_agent_logic(
    prompt_text="I want to eat chocolate, but too much calories. suggest alternate cheat meals that dont cheat my meal! i dont have cocoa"
)

NameError: name 'execute_agent_logic' is not defined